# NYC Reservoir data
    Scrapes the NYC water supply website for relevant data on reservoir levels and releases.
    Doesn't store data, just gets latest values.
    If NYC changes website format, this code will not work.
    
    #    Joel Plummer
    #    Geospatial Analyst
    #    Dewberry
    #    8401 Arlington Boulevard
    #    Fairfax, VA 22031
    #    jcplummer@dewberry.com

#### General method sources:
    # https://www.r-bloggers.com/reading-html-pages-in-r-for-text-processing/
    # http://zevross.com/blog/2015/05/19/scrape-website-data-with-the-new-r-package-rvest/

In [1]:
library(rvest)
library(dplyr)

Loading required package: xml2

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
# grab NYC reservoir water level webpage
inpath<-"http://www.nyc.gov/html/dep/html/drinking_water/maplevels_wide.shtml"
selector_names<-c("p")

# parse doc for paragraphs and convert to text
doc.text <- inpath %>% read_html() %>% html_nodes(selector_names) %>% html_text()
lvltab<-NULL

# get date from top paragraph
date<-doc.text[1]

# parse for colon which identifies data
s<-gregexpr(":",doc.text)

# parse through each paragrah and grab the data
for (i in 1:length(s)){
    if (!s[[i]][1]==-1){
        s2<-gregexpr("%",doc.text[i])
        name<-substr(doc.text[i],1,s[[i]][1])
        name<-sub(" Capacity:","",name)
        cap<-substr(doc.text[i],(s[[i]][1]+2),s2[[1]][1]-1)
        cur<-substr(doc.text[i],s[[i]][2]+2,nchar(doc.text[i]))    
        lvltab<-rbind(lvltab,c(date,name,cap,cur))
    }
}

ERROR: Error in do.call(method, list(parsed_selector)): could not find function "xpath_element"


In [4]:
# format table
lvltab<-data.frame(lvltab)
lvltab[,3]<-sapply(lvltab[,3],function(x) gsub("B gallons","",x))
curcaps<-apply(lvltab[,c(3,4)],1,function(x) round(as.numeric(x[1])*(as.numeric(x[2])/100),1))
lvltab<-cbind(lvltab,curcaps)
colnames(lvltab)<-c("Date","Name","Cap.(B gals)","Level(Pct)","Level(B gals)")
lvltab[,3]<-as.numeric(as.character(lvltab[,3]))
lvltab[,4]<-as.numeric(as.character(lvltab[,4]))

# calc attributes
nycTcap<-sum(as.numeric(lvltab[,3]),na.rm=TRUE)
nycCcap<-sum(as.numeric(lvltab[,5]),na.rm=TRUE)
nycCcapPCT<-round((nycCcap/nycTcap)*100,1)

delres<-c("Cannonsville Reservoir","Neversink Reservoir","Pepacton Reservoir")
deltab<-filter(lvltab, Name %in% delres)
delTcap<-sum(as.numeric(deltab[,3]),na.rm=TRUE)
delCcap<-sum(as.numeric(deltab[,5]),na.rm=TRUE)
delCcapPCT<-round((delCcap/delTcap)*100,1)

ERROR: Error in data.frame(lvltab): object 'lvltab' not found


In [5]:
########################
# grab NYC reservoir releases webpage

inpath<-"http://www.nyc.gov/html/dep/html/drinking_water/release_channel_levels.shtml"
selector_names<-c(".card-head",".card-mgd",".card-ntu","h3")
reltab<-NULL
# parse doc for NYC "card data" and convert to text
inpage<-inpath %>% read_html()
for (i in 1:length(selector_names)){
    res<-inpage %>% html_nodes(selector_names[i]) %>% html_text()
    if (i==4) res<-rep(res[1],dim(reltab)[2])
    reltab<-rbind(reltab,res)
}
# format table
reltab<-t(reltab)
reltab<-data.frame(reltab[,c(4,1,2,3)])
colnames(reltab)<-c("Date","Name","Release(MGD)","Turbidity(ntu)")
reltab[,3]<-sapply(reltab[,3],function(x) gsub("mgd","",x))
reltab[,4]<-sapply(reltab[,4],function(x) gsub("Turbidity ","",x))
reltab[,4]<-sapply(reltab[,4],function(x) gsub("ntu","",x))

reltab[,3]<-as.numeric(as.character(reltab[,3]))
reltab[,4]<-as.numeric(as.character(reltab[,4]))

lvltab
reltab

ERROR: Error in do.call(method, list(parsed_selector)): could not find function "xpath_class"
